In [4]:
!pip install langchain_core langchain_openai langchain_community langgraph

In [20]:
!pip install google-search-results

In [21]:
import os
import operator
from typing import TypedDict, Annotated, List
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper
from langgraph.graph import StateGraph, END

In [22]:
os.environ["SERPAPI_API_KEY"] = "f2200056b637942b33b7d52018ce41ff349e16158c805729d3628a9886ed19e8"

llm = ChatOpenAI(
    api_key="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",
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-chat-v3.1:free",
    temperature=0.2,
    streaming=True
)

In [23]:
# --- Tool Definition ---
# Instantiate the SerpAPI search tool.
search_tool = SerpAPIWrapper()

# --- State Definition ---
# This defines the "memory" or "state" that flows through the graph.
class ResearchState(TypedDict):
    topic: str
    explanation: str
    summary: str


In [24]:
def researcher_agent(state: ResearchState) -> dict:
    """
    This agent uses a web search tool to find information on a topic and then explains it.
    """
    print("---RESEARCHER (with SerpApi Web Search)---")
    topic = state["topic"]

    # Use the tool to search for information. The .run() method takes the query string.
    search_results = search_tool.run(topic)

    print(f"Search Results:\n{search_results}")

    prompt = ChatPromptTemplate.from_template(
        """You are a helpful research assistant. Based on the following search results,
        provide a brief, easy-to-understand explanation of the topic: {topic}.

        Search Results:
        {search_results}
        """
    )

    chain = prompt | llm
    result = chain.invoke({"topic": topic, "search_results": search_results})

    print(f"Researcher's Explanation:\n{result.content}")
    return {"explanation": result.content}

In [25]:
def summarizer_agent(state: ResearchState) -> dict:
    """
    This agent takes an explanation and summarizes it in one sentence.
    """
    print("---SUMMARIZER---")
    explanation = state["explanation"]

    prompt = ChatPromptTemplate.from_template(
        "You are a summarization expert. Condense the following text into a single, concise sentence:\n\n{explanation}"
    )

    chain = prompt | llm
    result = chain.invoke({"explanation": explanation})

    print(f"Summarizer's Output:\n{result.content}")
    return {"summary": result.content}

In [26]:
# --- Graph Definition ---

workflow = StateGraph(ResearchState)

# Add the agent functions as nodes
workflow.add_node("researcher", researcher_agent)
workflow.add_node("summarizer", summarizer_agent)

# Define the edges, which control the flow
workflow.add_edge("researcher", "summarizer")
workflow.add_edge("summarizer", END)

# Set the entry point
workflow.set_entry_point("researcher")

# Compile the graph
app = workflow.compile()

In [27]:
# --- Main Execution Block ---
if __name__ == "__main__":
    topic = input("Please enter a topic for the agents to research and summarize: ")

    inputs = {"topic": topic}
    final_state = app.invoke(inputs)

    print("\n--- FINAL SUMMARY ---")
    print(final_state['summary'])

Please enter a topic for the agents to research and summarize: IBM India
---RESEARCHER (with SerpApi Web Search)---
Search Results:
['Technology & Consulting. From next-generation AI to cutting edge hybrid cloud solutions to the deep expertise of IBM Consulting, IBM has what it takes to help ...', 'IBM India Private Limited is the Indian subsidiary of IBM. ... It has facilities in Ahmedabad, Bengaluru, Bhubaneshwar, Chennai, Coimbatore, Delhi, Gurgaon, ...', 'They laid off our entire department and offshored the labor to India where they could pay workers $2/hour instead of a living North American ...', "Here you'll find the jobs that best match your skills and interest. Use the filters to narrow down to what you're exactly looking for and apply.", 'For more than a century, IBM has been a global technology innovator, leading advances in AI, automation and hybrid cloud solutions that help businesses ...']
Researcher's Explanation:
Of course! Here is a simple explanation of IBM India bas

In [28]:
!pip install google-search-results